In [1]:
from diarization import diarization

Using device: cuda
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


speaker-embedding.onnx:   0%|          | 0.00/26.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [2]:
%%time
device = "cuda"

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [3]:
%%time
transcribation = diarization("/content/Встреча 7. Аудиозапись.mp3")

You have passed language=ru, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=ru.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


CPU times: user 4min 36s, sys: 1.06 s, total: 4min 37s
Wall time: 4min 44s


In [26]:
import json

In [25]:
with open('data.json', 'w') as f:
    json.dump(transcribation, f)

In [27]:
with open('data.json', 'r', encoding="utf-8") as f:
    data = json.load(f)

In [9]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [28]:
transcribation

{'0.03, 0.05': {'speaker': 'SPEAKER_01', 'text': 'С вами был Игорь Негода.'},
 '0.08, 32.35': {'speaker': 'SPEAKER_01',
  'text': ' Приветствую вас, уважаемый Алжа Собаевич, дорогие друзья. Мы рады вас видеть в доме правительства Российской Федерации. И прежде всего хотел бы лично поздравить вас с назначением премьер-министром Республики Казахстан. Для нас важно, что вы первый свой зарубежный визит начинаете с посещения Российской Федерации. Это в полной мере, на мой взгляд, отражает характер особых отношений между Российской Федерацией и Казахстаном.'},
 '32.87, 47.03': {'speaker': 'SPEAKER_01',
  'text': ' и эти отношения уверенно являются братским. Подтверждаю готовность к конструктивной работе совместной с вами по всей содержательной повестке нашего сотрудничества. Наша главная задача – обеспечить'},
 '47.18, 48.58': {'speaker': 'SPEAKER_01', 'text': ' Принятие всех решений.'},
 '48.87, 53.22': {'speaker': 'SPEAKER_01',
  'text': ' и договорённости, которые достигнуты нашими руково

In [47]:
%%time
from LLM import *
from Prompts import *
from TextTransformation import *

json_file = make_correction(gigachat, verificatiom_correctness_prompt, data) #сформировать корректный файл по словам, грамматике, правопсание
participants, text = diarisation_to_text(json_file) #функция превращения diarization.json в один большой текстовый файл для отправки в LLM
mapping = make_mapping(gigachat, make_speaker_mapping_prompt, text)
print(mapping)
questions = make_questions(gigachat, getting_question_prompt, text) #сбор обсуждаемых вопросов в рамках всего текста
detail_questions = make_detail_questions(gigachat, questions_details_prompt, text, questions) #сбор детального описание каждого вопроса
theme = make_theme(gigachat, make_theme_prompt, text)
tasks = make_tasks(gigachat, tasks_details_prompt, text)
final_dict = make_final_dict(theme, participants, detail_questions, tasks)
final_dict = replace_speaker(final_dict, mapping)

{[speaker_1:{''ФИО неизвестно", "Должность неизвестна" ]}, {{speaker_2:[''ФИО неизвестно", "Должность неизвестна" }]}
CPU times: user 65.6 ms, sys: 10.5 ms, total: 76.1 ms
Wall time: 38 s


In [50]:
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.shared import OxmlElement
from docx.oxml.ns import qn
import random
from datetime import timedelta, datetime


def insertHR(paragraph):
    # Устанавливаем форматирование для параграфа
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_before = Pt(0)  # Убираем отступ перед линией
    paragraph_format.space_after = Pt(0)   # Убираем отступ после линии

    p = paragraph._p  # p is the <w:p> XML element
    p.size = Pt(1)
    pPr = p.get_or_add_pPr()
    pBdr = OxmlElement('w:pBdr')
    pPr.insert_element_before(pBdr,
        'w:shd', 'w:tabs', 'w:suppressAutoHyphens', 'w:kinsoku', 'w:wordWrap',
        'w:overflowPunct', 'w:topLinePunct', 'w:autoSpaceDE', 'w:autoSpaceDN',
        'w:bidi', 'w:adjustRightInd', 'w:snapToGrid', 'w:spacing', 'w:ind',
        'w:contextualSpacing', 'w:mirrorIndents', 'w:suppressOverlap', 'w:jc',
        'w:textDirection', 'w:textAlignment', 'w:textboxTightWrap',
        'w:outlineLvl', 'w:divId', 'w:cnfStyle', 'w:rPr', 'w:sectPr',
        'w:pPrChange'
    )

    bottom = OxmlElement('w:bottom')
    bottom.set(qn('w:val'), 'single')
    bottom.set(qn('w:sz'), '6')
    bottom.set(qn('w:space'), '0')  # Уменьшение отступа линии
    bottom.set(qn('w:color'), 'auto')
    pBdr.append(bottom)

def create_protocol_doc_official(save_path, data, mapping):
    # Create a new Document
    doc = Document()

    style = doc.styles['Normal']
    font = style.font
    font.size = Pt(14)
    font.name = 'Times New Roman'

    # Title
    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run('ПРОТОКОЛ')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    insertHR(doc.add_paragraph())

    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run('Москва')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # doc.add_paragraph(f'от {data["creat_time"][:10]} №_____________').alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run(f'от {data["create_time"][:10]} №_____________')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

    # Add Participants
    doc.add_paragraph('Присутствовали:')

    for position, name in mapping.values():
        p = doc.add_paragraph(position + "  —  " + name)
        p.paragraph_format.left_indent = Pt(8)
        p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

    insertHR(doc.add_paragraph())
    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run('I. Обсуждаемые вопросы')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    insertHR(doc.add_paragraph())

    names = []
    # Add sections and questions
    for question in data['questions']:
        quest = question[list(question.keys())[0]]
        for item in question[list(question.keys())[0]]:
           if type(item['people'] == list):
              names += item['people']
           else:
             names += [item['people']]
    names = list(set(names))

    section_counter = 1
    start_time = datetime.strptime("00:00", "%H:%M")
    random_secs = random.uniform(1, 10)  # Generate a random float between 1 and 2
    random_minutes = random.uniform(0, 1)

    end_time = start_time + timedelta(minutes=random_secs) + timedelta(hours=random_minutes)
    start_time = end_time

    for question in data['questions']:
        section_title = f"{section_counter}. {list(question.keys())[0]}"

        if section_counter == 1:
            title_paragraph = doc.add_paragraph()
            title_run = title_paragraph.add_run(f'({", ".join(names)})')
            title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        title_paragraph = doc.add_paragraph()
        title_run = title_paragraph.add_run(section_title)
        title_run.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        # heading.font.name = 'Times New Roman'  # Set the font name
        # heading.font.color.rgb = RGBColor(0, 0, 0)
        section_counter += 1

        random_secs = random.uniform(1, 20)  # Generate a random float between 1 and 2
        random_minutes = random.uniform(1, 1.5) # Generate a random float between 1 and 2
        end_time = start_time + timedelta(minutes=random_secs) + timedelta(hours=random_minutes)

        time_paragraph = doc.add_paragraph()

        run = time_paragraph.add_run(f"Время: {start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')}")
        run.italic = True
        run.font.color.rgb = RGBColor(0, 0, 210)
        time_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

        start_time = end_time


        for item in question[list(question.keys())[0]]:
            decision = item['decision']
            people = ', '.join(item['people'])
            context = item['context']
            deadline = item['deadline']

            paragraph = doc.add_paragraph()
            first_part = "Контекст обсуждения:"
            second_part = f" {context}"
            run = paragraph.add_run(first_part)
            run.bold = True
            run2 = paragraph.add_run(second_part)
            run.italic = True
            run.font.color.rgb = RGBColor(0, 0, 210)
            run2.italic = True
            run2.font.color.rgb = RGBColor(0, 0, 210)
            # doc.add_paragraph(f"Контекст обсуждения: {context}")

    doc.save(save_path)

def create_protocol_doc_unofficial(save_path, data, mapping):
    # Create a new Document
    doc = Document()

    style = doc.styles['Normal']
    font = style.font
    font.size = Pt(14)
    font.name = 'Times New Roman'

    # Title
    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run('ПРОТОКОЛ')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER


    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run('Москва')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run(f'Дата совещания {data["create_time"][:10]}')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

    # Add Participants
    doc.add_paragraph('Присутствовали:')

    for position, name in mapping.values():
        p = doc.add_paragraph(position + "  —  " + name)
        p.paragraph_format.left_indent = Pt(8)
        p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

    title_paragraph3 = doc.add_paragraph()
    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run('Обсуждаемые вопросы')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    names = []
    # Add sections and questions
    for question in data['questions']:
        quest = question[list(question.keys())[0]]
        for item in question[list(question.keys())[0]]:
           if type(item['people'] == list):
              names += item['people']
           else:
             names += [item['people']]
    names = list(set(names))

    start_time = datetime.strptime("00:00", "%H:%M")
    random_secs = random.uniform(1, 10)  # Generate a random float between 1 and 2
    random_minutes = random.uniform(0, 1)

    end_time = start_time + timedelta(minutes=random_secs) + timedelta(hours=random_minutes)
    start_time = end_time

    section_counter = 1
    for question in data['questions']:
        section_title = f"{section_counter}. {list(question.keys())[0]}"

        if section_counter == 1:
            title_paragraph = doc.add_paragraph()
            title_run = title_paragraph.add_run(f'Активные участники: ({", ".join(names)})')
            title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        title_paragraph = doc.add_paragraph()
        title_run = title_paragraph.add_run(section_title)
        title_run.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        section_counter += 1

        for item in question[list(question.keys())[0]]:
            decision = item['decision']
            people = ', '.join(item['people'])
            context = item['context']
            deadline = item['deadline']

            paragraph = doc.add_paragraph()
            first_part = "Контекст обсуждения:"
            second_part = f" {context}"
            run = paragraph.add_run(first_part)
            run.bold = True
            run2 = paragraph.add_run(second_part)
            run.italic = True
            run.font.color.rgb = RGBColor(0, 0, 210)
            run2.italic = True
            run2.font.color.rgb = RGBColor(0, 0, 210)
            # doc.add_paragraph(f"Контекст обсуждения: {context}")

            random_secs = random.uniform(1, 20)  # Generate a random float between 1 and 2
            random_minutes = random.uniform(1, 1.5) # Generate a random float between 1 and 2
            end_time = start_time + timedelta(minutes=random_secs) + timedelta(hours=random_minutes)

            time_paragraph = doc.add_paragraph()

            run = time_paragraph.add_run(f"Время: {start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')}")
            run.italic = True
            run.font.color.rgb = RGBColor(0, 0, 210)
            time_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

            start_time = end_time

    title_paragraph5 = doc.add_paragraph()
    title_paragraph = doc.add_paragraph()
    title_run = title_paragraph.add_run('Назначенные задачи')
    title_run.bold = True
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER


    for task in data['tasks']['tasks']:

        task_title = task['task']
        responsible = task['responsible']
        deadline = task['deadline']

        doc.add_paragraph(f"Задача: {task_title}")

        title_paragraph = doc.add_paragraph()
        title_run = title_paragraph.add_run('Ответственный: ')
        title_run.italic = True
        title_run.bold = True
        title_run2 = title_paragraph.add_run(f'{responsible}')
        title_run2.italic = True
        title_run2.font.color.rgb = RGBColor(0, 0, 210)
        title_run.font.color.rgb = RGBColor(0, 0, 210)

        if len(deadline) > 2:
          doc.add_paragraph(f"Срок: {deadline}")

    doc.save(save_path)

In [52]:
%%time
save_path = 'protocol.docx'
create_protocol_doc_unofficial(save_path, tere, mapping)

CPU times: user 13.9 ms, sys: 1.96 ms, total: 15.8 ms
Wall time: 7.35 s
